# Business Project

Introduction:

New York is the most populous city in the United States. Also, the heart of the global financial market and This project will analyze which Hotel would be appropriate for the tourist to have nightlife on the trip to New York. For those who want to have a nice experience of hostel service after their nightlife, the content of this report is also suitable. This project will analyze the location of the convenience store, bar, and hostel.

Data section:

The report will use the data from the Foursquare API.the data will include the location of the hotel, bar, and convenience store in NewYork City. Then, The hostel, bar, and convenience store will be mapped by the folium library. The map visualizes the location of the venues above that will demonstrate the distribution of the bar and hotel. The hostel close to the bar and the convenience store would be selected. The selected group of hotels would be compared by the rating from Foursquare. Finally, the recommodation will be made by these process.

Methodology section:

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [21]:
address = 'New York'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [20]:
CLIENT_ID = '5G34KH1WV3TBA5XW2PTG0WORN3NE5WETRKTE1MFO41XCVSTZ' # your Foursquare ID
CLIENT_SECRET = '2STZQANK0KBEMBEFBJNUUMRTVL3DYGTJ4H3ZWEWP41PQCSHS' # your Foursquare Secret
VERSION = '20200522' # Foursquare API version
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5G34KH1WV3TBA5XW2PTG0WORN3NE5WETRKTE1MFO41XCVSTZ
CLIENT_SECRET:2STZQANK0KBEMBEFBJNUUMRTVL3DYGTJ4H3ZWEWP41PQCSHS


In [22]:
search_query = ''
categoryId='4bf58dd8d48988d116941735' #id for bar

radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT, categoryId)

search_query1 = ''
categoryId1='4bf58dd8d48988d1fa931735' # for hostel
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT, categoryId1)

search_query2 = ''
categoryId2='4d954b0ea243a5684a65b473' # for Convenience Store
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT, categoryId2)


In [23]:
results = requests.get(url).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']


# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe1 = json_normalize(venues1)
dataframe2 = json_normalize(venues2)


dataframe.shape,
dataframe1.shape,
dataframe2.shape

(25, 18)

In [24]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]



# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]






In [25]:
# create map of New York using latitude and longitude values
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15
                    ) 

# add markers to map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 
    
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.name):folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 
    

for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.name):folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 
    
    
venues_map


Central Harlem 10026, 10027, 10030, 10037, 10039
Chelsea and Clinton 10001, 10011, 10018, 10019, 10020, 10036
East Harlem 10029, 10035
Gramercy Park and Murray Hill	10010, 10016, 10017, 10022
Greenwich Village and Soho	10012, 10013, 10014
Lower Manhattan	10004, 10005, 10006, 10007, 10038, 10280
Lower East Side	10002, 10003, 10009
Upper East Side	10021, 10028, 10044, 10065, 10075, 10128
Upper West Side	10023, 10024, 10025
Inwood and Washington Heights	10031, 10032, 10033, 10034, 10040

Find the rating of certain hostel

In [47]:
dataframe_filtered2.to_csv('cs.csv')
df2=pd.read_csv('cs.csv')
df22 = df2.filter(['name','postalCode','id'], axis=1)
df22.rename(columns={"postalCode":"district"},inplace=True)
df22.replace(to_replace = [10004, 10005, 10006, 10007, 10038, 10280] ,value ="Lower Manhattan", inplace = True) 
df22.replace(to_replace = [10012, 10013, 10014] ,value ="Greenwich Village and Soho", inplace = True) 
df22.replace(to_replace = [10278] ,value ="Tribeca", inplace = True) 
df22[['district']] = df22[['district']].fillna(value="Lower Manhattan")

dataframe_filtered.to_csv('cs.csv')
df=pd.read_csv('cs.csv')
df = df.filter(['name','postalCode','id'], axis=1)
df.rename(columns={"postalCode":"district"},inplace=True)
df.replace(to_replace = [10004, 10005, 10006, 10007, 10038, 10280] ,value ="Lower Manhattan", inplace = True) 
df.replace(to_replace = [10012, 10013, 10014] ,value ="Greenwich Village and Soho", inplace = True) 
df.replace(to_replace = [10002, 10003, 10009] ,value ="Lower East Side", inplace = True) 
df.replace(to_replace = [10278,10279,10271] ,value ="Tribeca", inplace = True) 
df[['district']] = df[['district']].fillna(value="Lower Manhattan")

dataframe_filtered1.to_csv('cs.csv')
df1=pd.read_csv('cs.csv')
df1 = df1.filter(['name','address','postalCode','id'], axis=1)
df1.rename(columns={"postalCode":"district"},inplace=True)
df1.replace(to_replace = [10004, 10005, 10006, 10007, 10038, 10280] ,value ="Lower Manhattan", inplace = True) 
df1.replace(to_replace = [10012, 10013, 10014] ,value ="Greenwich Village and Soho", inplace = True) 
df1.replace(to_replace = [10002, 10003, 10009] ,value ="Lower East Side", inplace = True) 
df1.replace(to_replace = [10278] ,value ="Tribeca", inplace = True)
df1[['district']] = df1[['district']].fillna(value="Lower Manhattan")
df1

,name,address,district,id
0,"The Beekman, A Thompson Hotel",123 Nassau St,Lower Manhattan,56d8c0f8498edb854f926e6a
1,Moxy NYC Downtown,26 Ann St,Lower Manhattan,5bd4cc646adbf5002c1ced5c
2,Aloft Manhattan Downtown - Financial District,49-53 Ann St,Lower Manhattan,547dc381498ec37f84f01439
3,Millennium Hilton,55 Church St,Lower Manhattan,510ad6e2e4b0a2ecec03b322
4,Duane Street Hotel,130 Duane St,Greenwich Village and Soho,4ae77379f964a5206cab21e3
5,Hampton Inn by Hilton,320 Pearl St,Lower Manhattan,4a8ed915f964a520e11220e3
6,Holiday Inn New York City - Wall Street,51 Nassau St,Lower Manhattan,4b0017f7f964a520bb3a22e3
7,Artezen Hotel,24 John St,Lower Manhattan,5a91fa58e679bc57ca94db72
8,Four Seasons Hotel New York Downtown,27 Barclay St,Lower Manhattan,57c640ad498e74977f98372f
9,Four Seasons,30 Park Place,Lower Manhattan,57c06f2e498e1de2b349c99e


10	Fun City Times Square	215 Broadway	Lower Manhattan	 4c6bad15599fbe9a412b933f
24	The Frederick Hotel	95 W Broadway	Lower Manhattan	 59e391c3e1f22816d1b09cf5
25	The Frederick Hotel	95 West Broadway	Lower Manhattan	 4bed7bbebac3c9b60157fee9
22	Cosmopolitan Hotel - TriBeCa	95 W Broadway	Lower Manhattan	 4a3a9952f964a52001a01fe3
16	Smythe Thompson Hotel	86 W Broadway	Lower Manhattan	 52855bf511d21362ef6f329f
13	Aka Tribeca	85 W Broadway	Lower Manhattan	 5c982c89a22db7003842c21a
18	Smyth Hotel	85 W Broadway	Lower Manhattan	 49efcc88f964a52006691fe3
19	Residence Inn by Marriott New York Downtown Ma...	170 Broadway	Lower Manhattan	 54aed577498ee5fc196739f7


In [49]:
venue_id = '4c6bad15599fbe9a412b933f',  # ID of Fun City Times Square
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '4e9481cc5c5c9bf8ad96ca03',  # ID of Acquis Executive Lounge	
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '59e391c3e1f22816d1b09cf5' # ID of The Frederick Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The Frederick Hotel:',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4bed7bbebac3c9b60157fee9' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '4a3a9952f964a52001a01fe3' # ID of Cosmopolitan Hotel - TriBeCa
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Cosmopolitan Hotel - TriBeCa:',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id ='52855bf511d21362ef6f329f' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '5c982c89a22db7003842c21a' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = ' 49efcc88f964a52006691fe3 ' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '54aed577498ee5fc196739f7' # ID of Residence Inn by Residence Inn by Marriott New York Downtown Manhattan
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('Residence Inn by Marriott New York Downtown Manhattan',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    


This venue has not been rated yet.
This venue has not been rated yet.
The Frederick Hotel: 8.2
This venue has not been rated yet.
Cosmopolitan Hotel - TriBeCa: 6.1
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
Residence Inn by Marriott New York Downtown Manhattan 8.0


Results section:

it can only return the rating of The Frederick Hotel,Residence Inn by Marriott and Cosmopolitan Hotel - TriBeCa.The Frederick Hotel and Residence Inn by Marriott both rated at 8. both of these hotels have the convenience stole nearby.

Discussion section:

it can be observed that in the particular area, The hotel are quite close to the bar. it may be due to the complement relationship between these 2 services.

this report is to find out which hotels are suitable for the nightlife and have a high quality of service. it can be found by sorting the hotel which is close to the bar area and have at least 1 convenience store. After the sorting part, The rating of the hostel will be compared.

we can find that the hostels above have high rating, close to the bar area and the convience store. it could be a suitable choice for the tourist who want to have the nightlife.

difficulty:
Data searching and clearing is time-consuming . My original idea is to finds out the distributon of the Michelin resturant in Hong Kong/Tokyo. However, there is not the categories of “Michelin” in the foursquare. The
 .The limit of the data from the foursquare.com limit the scope of geographic area can be analysed.



Conclusion:

All in All, this project is finish. if someone follow the suggestion of this report which i suppose no one will do that, please tell me about the experience. there are also some new ideas after doing this report. for example, is there the positive relation between the number of the hostel and the bar. does the hostel close to the bar will have higher attendance than the  hotel routine from the bar? i think it is the interesting question to investigate. i hope you would like this.
